In [1]:
from transformers import pipeline, set_seed
import torch
import tensorflow as tf
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, I didn't want to work with another language like Rust. I'm making more time for Rust. I just"},
 {'generated_text': 'Hello, I\'m a language model, and I think this is a great time to look at how languages and their dependencies do it," he said.'},
 {'generated_text': "Hello, I'm a language model, and writing one is like writing a dictionary. I can put any number of objects together, and it keeps the"},
 {'generated_text': "Hello, I'm a language model, I'm a grammar model\n\nLinguistics [ edit ]\n\nThe term linguistics refers to the"},
 {'generated_text': "Hello, I'm a language model, not a system programming language. The language we were writing in Java was very basic and it was pretty much a"}]

In [2]:

from sklearn.model_selection import train_test_split
import re
all_data = ["I don't know --",
"You don't know?",
"Eric asked if I could cover for him, I said yeah; I figured we could use the cash --",
"We talked about this -- I told you not to take shifts on nights I'm tending, I specifically told you --",
"He was over at the Wheelers' all day. I'm sure he just stayed over.",
"I can't believe it.",
"Chief, Joyce is very upset and --",
"What have we talked about? Morning is a time for coffee and contemplation.",
"He asked --! ",
"Hey! What’d I just say! One at a damn time!",
"Mom, you need to stay calm.",
"I’m calm.",
"Hey, you’ve reached Lonnie, I’m not here at the moment but …",
"Lonnie, some teenage whore just hung up on me. You don’t call me back in the next hour I’ll report you for not paying child support I swear to God I will and I’ll make sure you rot in jail where you belong -- ",
"So maybe he came back here.",
"So -- what? You think I haven’t checked my own house?!"]

def build_text_files(the_data, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in the_data:
            print(texts)
            summary = re.sub(r"\s", " ", texts)
            data += summary + "  "
#     print(data)
    f.write(data)

train, test = train_test_split(all_data,test_size=0.15) 


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

# print(train)
# print(test)


So maybe he came back here.
Lonnie, some teenage whore just hung up on me. You don’t call me back in the next hour I’ll report you for not paying child support I swear to God I will and I’ll make sure you rot in jail where you belong -- 
I’m calm.
He asked --! 
Hey! What’d I just say! One at a damn time!
Eric asked if I could cover for him, I said yeah; I figured we could use the cash --
So -- what? You think I haven’t checked my own house?!
He was over at the Wheelers' all day. I'm sure he just stayed over.
What have we talked about? Morning is a time for coffee and contemplation.
Mom, you need to stay calm.
Hey, you’ve reached Lonnie, I’m not here at the moment but …
We talked about this -- I told you not to take shifts on nights I'm tending, I specifically told you --
Chief, Joyce is very upset and --
I don't know --
You don't know?
I can't believe it.
Train dataset length: 13
Test dataset length: 3


In [3]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
from transformers import TextDataset,DataCollatorForLanguageModeling

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/lexiryan/miniconda3/envs/cs338/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [4]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained('gpt2')




/Users/lexiryan/miniconda3/envs/cs338/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [5]:
training_args = TrainingArguments(
    output_dir="./gpt2-sandbox", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [6]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=4.14202880859375, metrics={'train_runtime': 12.9101, 'train_samples_per_second': 0.232, 'total_flos': 573418635264, 'epoch': 3.0})

In [7]:
trainer.save_model()

In [8]:
from transformers import pipeline

gen = pipeline('text-generation',model='./gpt2-sandbox', tokenizer='gpt2',config={'max_length':800})


In [9]:
gen("I think")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I think I could live with it," Mr. Soto said.\n\nHe will soon be retiring from the Army.\n\n"Not as a politician, but as a person," he added. "My job is to do what I can'}]